# This is

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).


In [1]:
import pandas as pd
import glob
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

In [6]:
# Ruta a los archivos CSV del dataset
# Verificar el directorio actual de ejecución
print("Directorio actual:", os.getcwd())

ruta_archivos = 'data/*.csv'

# Cargar todos los archivos CSV en un solo DataFrame
archivos = glob.glob(ruta_archivos)
dataframes = [pd.read_csv(archivo) for archivo in archivos]
df = pd.concat(dataframes, ignore_index=True)

# Mostrar información básica del dataset
print(df.info())
print(df.head())
# Eliminar columnas irrelevantes (como Timestamp si existe)



Directorio actual: U:\Proyectos\Regionathon2024
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830743 entries, 0 to 2830742
Data columns (total 79 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0    Destination Port             int64  
 1    Flow Duration                int64  
 2    Total Fwd Packets            int64  
 3    Total Backward Packets       int64  
 4   Total Length of Fwd Packets   int64  
 5    Total Length of Bwd Packets  int64  
 6    Fwd Packet Length Max        int64  
 7    Fwd Packet Length Min        int64  
 8    Fwd Packet Length Mean       float64
 9    Fwd Packet Length Std        float64
 10  Bwd Packet Length Max         int64  
 11   Bwd Packet Length Min        int64  
 12   Bwd Packet Length Mean       float64
 13   Bwd Packet Length Std        float64
 14  Flow Bytes/s                  float64
 15   Flow Packets/s               float64
 16   Flow IAT Mean                float64
 17   Flow IAT Std    

In [7]:
print("Dimensiones del dataset:", df.shape)
print("Nombres de columnas:", df.columns)

Dimensiones del dataset: (2830743, 79)
Nombres de columnas: Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean

In [8]:
# Limpiar nombres de columnas
df.columns = df.columns.str.strip()

# Verificar que la columna 'Label' exista
if 'Label' not in df.columns:
    raise ValueError("La columna 'Label' no existe en el DataFrame.")

# Inspeccionar valores únicos en 'Label'
print("Valores únicos en Label antes de limpiar:", df['Label'].unique())

# Limpiar valores en 'Label'
df['Label'] = df['Label'].str.strip()

# Inspeccionar valores únicos después de limpiar
print("Valores únicos en Label después de limpiar:", df['Label'].unique())

# Codificar etiquetas (LabelEncoder)
label_encoder = LabelEncoder()
df['Label'] = label_encoder.fit_transform(df['Label'])

# Mostrar clases codificadas
print("Clases codificadas:", dict(enumerate(label_encoder.classes_)))

# Dividir características (X) y etiquetas (y)
X = df.drop(columns=['Label'])  # Características
y = df['Label']  # Etiquetas

# Manejar valores problemáticos en X
# Reemplazar valores infinitos por NaN
X.replace([np.inf, -np.inf], np.nan, inplace=True)

# Rellenar valores NaN con la mediana de cada columna
X.fillna(X.median(), inplace=True)

# Validar que no existan más valores problemáticos
assert not X.isnull().any().any(), "Todavía hay valores NaN en las características."
assert not (X == float('inf')).any().any(), "Todavía hay valores infinitos en las características."

# Normalización de características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


Valores únicos en Label antes de limpiar: ['BENIGN' 'DDoS' 'PortScan' 'Bot' 'Infiltration'
 'Web Attack � Brute Force' 'Web Attack � XSS'
 'Web Attack � Sql Injection' 'FTP-Patator' 'SSH-Patator' 'DoS slowloris'
 'DoS Slowhttptest' 'DoS Hulk' 'DoS GoldenEye' 'Heartbleed']
Valores únicos en Label después de limpiar: ['BENIGN' 'DDoS' 'PortScan' 'Bot' 'Infiltration'
 'Web Attack � Brute Force' 'Web Attack � XSS'
 'Web Attack � Sql Injection' 'FTP-Patator' 'SSH-Patator' 'DoS slowloris'
 'DoS Slowhttptest' 'DoS Hulk' 'DoS GoldenEye' 'Heartbleed']
Clases codificadas: {0: 'BENIGN', 1: 'Bot', 2: 'DDoS', 3: 'DoS GoldenEye', 4: 'DoS Hulk', 5: 'DoS Slowhttptest', 6: 'DoS slowloris', 7: 'FTP-Patator', 8: 'Heartbleed', 9: 'Infiltration', 10: 'PortScan', 11: 'SSH-Patator', 12: 'Web Attack � Brute Force', 13: 'Web Attack � Sql Injection', 14: 'Web Attack � XSS'}


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)



In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tqdm import tqdm

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Barra de progreso para seguimiento del entrenamiento
progress_bar = tqdm(total=100, desc="Entrenando Random Forest")
for i in range(1, 101):
    rf_model.set_params(n_estimators=i)
    rf_model.fit(X_train, y_train)
    progress_bar.update(1)
progress_bar.close()

# Predicción en el conjunto de prueba
y_pred = rf_model.predict(X_test)

# Decodificar predicciones y etiquetas reales
y_pred_labels = label_encoder.inverse_transform(y_pred)
y_test_labels = label_encoder.inverse_transform(y_test)

# Generar el reporte de clasificación
print("\nClassification Report:\n", classification_report(y_test_labels, y_pred_labels))
print("Confusion Matrix:\n", confusion_matrix(y_test_labels, y_pred_labels))


Entrenando Random Forest: 100%|██████████| 100/100 [12:56:26<00:00, 465.87s/it] 



Classification Report:
                             precision    recall  f1-score   support

                    BENIGN       1.00      1.00      1.00    681572
                       Bot       0.91      0.70      0.79       600
                      DDoS       1.00      1.00      1.00     38301
             DoS GoldenEye       1.00      1.00      1.00      3099
                  DoS Hulk       1.00      1.00      1.00     69516
          DoS Slowhttptest       0.99      1.00      0.99      1621
             DoS slowloris       1.00      1.00      1.00      1761
               FTP-Patator       1.00      1.00      1.00      2422
                Heartbleed       1.00      0.50      0.67         2
              Infiltration       1.00      0.88      0.93         8
                  PortScan       0.99      1.00      1.00     47894
               SSH-Patator       1.00      1.00      1.00      1762
  Web Attack � Brute Force       0.75      0.90      0.82       472
Web Attack � Sql Injec

In [11]:
# Convertir las etiquetas en categorías numéricas
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['Label'] = label_encoder.fit_transform(df['Label'])


In [12]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Codificar etiquetas
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Predecir etiquetas con el modelo
y_pred = rf_model.predict(X_test)

# Convertir las clases a cadenas de texto
target_names = [str(cls) for cls in label_encoder.classes_]

# Generar el reporte de clasificación
print("Classification Report (Multiclass):\n", classification_report(y_test, y_pred, target_names=target_names))

Classification Report (Multiclass):
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    681572
           1       0.91      0.70      0.79       600
           2       1.00      1.00      1.00     38301
           3       1.00      1.00      1.00      3099
           4       1.00      1.00      1.00     69516
           5       0.99      1.00      0.99      1621
           6       1.00      1.00      1.00      1761
           7       1.00      1.00      1.00      2422
           8       1.00      0.50      0.67         2
           9       1.00      0.88      0.93         8
          10       0.99      1.00      1.00     47894
          11       1.00      1.00      1.00      1762
          12       0.75      0.90      0.82       472
          13       0.50      0.11      0.18         9
          14       0.48      0.20      0.28       184

    accuracy                           1.00    849223
   macro avg       0.91      0.82      0.84

In [15]:
import joblib

# Guardar el modelo

joblib.dump(rf_model, 'Back_Regionathon2024/modelsMod/random_forest_model_con_nombre_ataque.pkl')
joblib.dump(scaler, 'Back_Regionathon2024/modelsMod/scaler_nombre_ataque.pkl')

print("Modelo y escalador guardados correctamente.")


Modelo y escalador guardados correctamente.


In [ ]:
## Prueba real

# Ejemplo de predicción en tiempo real
nueva_muestra = X_test[0].reshape(1, -1)  # Usamos una muestra del conjunto de prueba
nueva_prediccion = rf_model.predict(nueva_muestra)
tipo_ataque = label_encoder.inverse_transform(nueva_prediccion)
print("\nPredicción para nueva muestra:")
print("El tipo de ataque es:", tipo_ataque[0])